In [19]:
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from kneed import KneeLocator
import pandas as pd
import pickle
from sklearn import preprocessing


# Load the dataset
data = pd.read_csv("train.csv")

def prepo(data):
# Handling Missing values
    data.dropna(inplace=True)

    # Label Encoding
    label_encoder = preprocessing.LabelEncoder()
    cat_col = data.select_dtypes(include="object").columns
    for i in cat_col:
        data[i]= label_encoder.fit_transform(data[i])
# Choose the features you want to use for clustering and modeling
    X = data.drop("satisfaction",axis=1)
    y = data["satisfaction"]

    return X,y
X,y = prepo(data=data)




# Use the kneed library to find the optimal number of clusters
sse = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X)
    sse.append(kmeans.inertia_)
kl = KneeLocator(range(1, 11), sse, curve="convex", direction="decreasing")
n_clusters = kl.elbow

# print(clusters.shape, X.shape)
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Cluster the data using k-means
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
clusters = kmeans.fit_predict(X_train)

# Train a separate model for each cluster and save them
for i in range(n_clusters):
    # Get the instances assigned to this cluster
    cluster_mask = clusters == i
    X_cluster = X_train[cluster_mask]
    y_cluster = y_train[cluster_mask]
    
    # Train a linear regression model on the cluster
    model = RandomForestClassifier()
    model.fit(X_cluster, y_cluster)
    
    # Save the trained model using pickle
    with open(f"model_{i}.pickle", "wb") as f:
        pickle.dump(model, f)



C:\Users\koush\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\koush\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\koush\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\U

(82875,) (103594, 24)


C:\Users\koush\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [29]:
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score

In [30]:
# Load the saved models and make predictions on new data
X_new = pd.read_csv("test.csv")
X,y = prepo(X_new)
for i in range(n_clusters):
    # Load the trained model
    with open(f"model_{i}.pickle", "rb") as f:
        model = pickle.load(f)
    
    # Get the instances assigned to this cluster
    # cluster_mask = kmeans.predict(X) == i
    # X_cluster_new = X[cluster_mask]
    
    # Make predictions using the model and the instances assigned to the cluster
    y_pred = model.predict(X)
    print(f"Predictions for cluster {i}: {(y_pred==y).sum()}")

    report = classification_report(y,y_pred)
    print(report)
    print(precision_score(y,y_pred))

Predictions for cluster 0: 24772
              precision    recall  f1-score   support

           0       0.95      0.97      0.96     14528
           1       0.96      0.94      0.95     11365

    accuracy                           0.96     25893
   macro avg       0.96      0.95      0.96     25893
weighted avg       0.96      0.96      0.96     25893

0.9641174338528452
Predictions for cluster 1: 24327
              precision    recall  f1-score   support

           0       0.91      0.98      0.95     14528
           1       0.98      0.88      0.93     11365

    accuracy                           0.94     25893
   macro avg       0.95      0.93      0.94     25893
weighted avg       0.94      0.94      0.94     25893

0.978420076164437
Predictions for cluster 2: 24785
              precision    recall  f1-score   support

           0       0.96      0.97      0.96     14528
           1       0.96      0.94      0.95     11365

    accuracy                           0.96   